<a href="https://colab.research.google.com/github/JS-DS-coder/CienciaDeDatos2021/blob/master/parte2SL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parte 2 - Deployment
Jairo Salazar


In [64]:
import joblib
import os.path
from os import path
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
import tensorflow as tf

if tf.__version__.startswith("2."):
  import tensorflow.compat.v1 as tf
  tf.compat.v1.disable_v2_behavior()
  tf.compat.v1.disable_eager_execution()
  print("Enabled compatitility to tf1.x")



if path.exists('/content/logs') == False:
  os.mkdir('/content/logs')

if path.exists('/content/logs/experimentos') == False:
  os.mkdir('/content/logs/experimentos')

if path.exists('/content/logs/modelos') == False:
  os.mkdir('/content/logs/modelos')

!pwd
!ls

Instructions for updating:
non-resource variables are not supported in the long term
Enabled compatitility to tf1.x
/content
logs  modelos  sample_data


In [9]:
DT = joblib.load("/content/logs/modelos/DecisionTree_Age_passenger_sex_IsAlone_FamilySize_SibSp.pkl")
NB = joblib.load("/content/logs/modelos/NaiveBayes_passenger_sex_IsAlone_FamiliySize_SibSp_GaussianNB.pkl")
SVM= joblib.load("/content/logs/modelos/SVM_passenger_sex_Has_Cabin_FamilySize.pkl")
SVM

GridSearchCV(cv=10, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=1,
             param_grid={'C': [1, 2, 5, 10, 20, 50]}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=True, scoring='accuracy',
             verbose=0)

In [60]:
def Transformacion(ds):
  #se quitan los NAs y se reemplaza con A
  ds["Embarked"].fillna("A", inplace = True)
  ds["Age"].fillna(0, inplace = True)
  #Se crea dos nuevas variables numericas.la primera nos indicara el numero de familiares en el barco, la segunda
  #es si viaja solo o no.
  ds["Age"] = int(ds["Age"])
  ds["Fare"] = float(ds["Fare"])
  ds['FamilySize'] = int(dataset['SibSp']) + int(dataset['Parch']) + 1
  ds['IsAlone'] = 0
  ds.loc[ds['FamilySize'] == 1, 'IsAlone'] = 1
  ds.loc[ds['Age'] <= 16, 'Age'] 					       = 0
  ds.loc[(ds['Age'] > 16) & (ds['Age'] <= 32), 'Age'] = 1
  ds.loc[(ds['Age'] > 32) & (ds['Age'] <= 48), 'Age'] = 2
  ds.loc[(ds['Age'] > 48) & (ds['Age'] <= 64), 'Age'] = 3
  ds.loc[ ds['Age'] > 64, 'Age'] = 4

  ds.loc[ dataset['Fare'] <= 7.91, 'Fare'] 						        = 0
  ds.loc[(dataset['Fare'] > 7.91) & (ds['Fare'] <= 14.454), 'Fare'] = 1
  ds.loc[(dataset['Fare'] > 14.454) & (ds['Fare'] <= 31), 'Fare']   = 2
  ds.loc[ dataset['Fare'] > 31, 'Fare'] 							        = 3
  ds['Fare'] = ds['Fare'].astype(int)

  cleanup_nums = {"passenger_sex": {"M": 1, "F": 0},
                 "passenger_class": {"Lower": 0, "Middle": 1,"Upper": 2},
                  "Embarked": {"S": 0, "C": 1,"Q": 2,"A": 3}
                  }
  ds = ds.replace(cleanup_nums) 
  ds['Has_Cabin'] = ds["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
  return ds

#Funcion que consulta el Registro a evaluar:
def crearRegistro():
 # 'PassengerId', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare','Cabin','Embarked','passenger_class', 'passenger_sex', 'passenger_survived'

  Passengerid = input ("Enter Passegerid :") 
  Name = input ("Enter Name :") 
  Age = float(input("Enter Age: "))
  SibSp =  int(input("Enter SibSp: "))
  Parch = int(input("Enter Parch:"))
  Ticket= input("Enter Ticket:")
  Fare = float(input("Enter Fare:"))
  Cabin = input("Enter Cabin:")
  Embarked = input("Enter Embarked:")
  passenger_class= input("Enter passenger_class:")
  passenger_sex= input("Enter passenger_sex:")
  df = pd.DataFrame(columns = ['PassengerId', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare','Cabin','Embarked','passenger_class', 'passenger_sex'])
  print(df)
  
  # append rows to an empty DataFrame
  df = df.append({'PassengerId' : int(Passengerid), 'Name' : Name, 'Age' : Age,'SibSp':int(SibSp),'Parch' :int(Parch),'Ticket':Ticket,'Fare':float(Fare),'Cabin':Cabin,'Embarked':Embarked,'passenger_sex':passenger_sex,'passenger_class':passenger_class},
                ignore_index = True)
  
  #return (df)
  return Transformacion(df)





Enter Passegerid :2
Enter Name :franco, miss ana
Enter Age: 32
Enter SibSp: 0
Enter Parch:1
Enter Ticket:SA-32
Enter Fare:10.21
Enter Cabin:CA-21
Enter Embarked:S
Enter passenger_class:Upper
Enter passenger_sex:F
Empty DataFrame
Columns: [PassengerId, Name, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked, passenger_class, passenger_sex]
Index: []


In [50]:
dataset

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,FamilySize,IsAlone,Has_Cabin
0,1,"Salazar, Sir Jairo",2,2,1,SA32,0,,2,1,1,4,0,1


In [80]:


def Eval(ds):
  #Desicion Tree
  c=["Age","passenger_sex","IsAlone","FamilySize","SibSp"]
  x = ds[c]

  yhatDT = DT.predict(x)
 
  #SVM
  c=["Age","passenger_sex","Has_Cabin","FamilySize"]
  x=ds[c]

  yhatSVM = SVM.predict(x)
  #Naive Bayes
  c=["Age","passenger_sex","IsAlone","FamilySize","SibSp"]
  x = ds[c]
  
  yhatNB = NB.predict(x)

  df = pd.DataFrame(columns = ['YhatDT', 'YhatSVM', 'YhatNB', 'YhatRegLog'])
  print(df)
  # REgresion logistica
  #YhatRegLog=regresionlogistica(ds)
  # append rows to an empty DataFrame
  df = df.append({'YhatDT' : yhatDT, 'YhatSVM' : yhatSVM, 'YhatNB' : yhatNB},
                ignore_index = True)
  
  return df


#Transformacion(dataset)

# Caso 1 

Datos:

In [88]:
dataset = crearRegistro()
dataset

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,FamilySize,IsAlone,Has_Cabin
0,2,"franco, miss ana",1,0,1,SA-32,0,CA-21,0,2,0,4,0,1


In [89]:
ds = Eval(dataset)
print('DATOS DE SALIDA: ')
ds
#print('Resultados: ',ds)

Empty DataFrame
Columns: [YhatDT, YhatSVM, YhatNB, YhatRegLog]
Index: []
DATOS DE SALIDA: 


,YhatDT,YhatSVM,YhatNB,YhatRegLog
0,[1],[1],[1],NaN


# Caso 2

In [90]:
dataset = crearRegistro()
dataset

Enter Passegerid :2
Enter Name :Salazar, Ing Jairo
Enter Age: 43
Enter SibSp: 0
Enter Parch:0
Enter Ticket:SA-21
Enter Fare:30.21
Enter Cabin:C
Enter Embarked:S
Enter passenger_class:Upper
Enter passenger_sex:M
Empty DataFrame
Columns: [PassengerId, Name, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked, passenger_class, passenger_sex]
Index: []


,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,FamilySize,IsAlone,Has_Cabin
0,2,"Salazar, Ing Jairo",2,0,0,SA-21,0,C,0,2,1,2,0,1


In [91]:
ds = Eval(dataset)
print('DATOS DE SALIDA: ')
ds

Empty DataFrame
Columns: [YhatDT, YhatSVM, YhatNB, YhatRegLog]
Index: []
DATOS DE SALIDA: 


,YhatDT,YhatSVM,YhatNB,YhatRegLog
0,[0],[0],[0],NaN


# Caso 3

In [92]:
dataset = crearRegistro()
dataset

Enter Passegerid :3
Enter Name :Salazar, Rodrigo
Enter Age: 5
Enter SibSp: 0
Enter Parch:0
Enter Ticket:SA021
Enter Fare:10.21
Enter Cabin:CA
Enter Embarked:S
Enter passenger_class:Middle
Enter passenger_sex:M
Empty DataFrame
Columns: [PassengerId, Name, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked, passenger_class, passenger_sex]
Index: []


,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,FamilySize,IsAlone,Has_Cabin
0,3,"Salazar, Rodrigo",0,0,0,SA021,0,CA,0,1,1,1,1,1


In [93]:
ds = Eval(dataset)
print('DATOS DE SALIDA: ')
ds

Empty DataFrame
Columns: [YhatDT, YhatSVM, YhatNB, YhatRegLog]
Index: []
DATOS DE SALIDA: 


,YhatDT,YhatSVM,YhatNB,YhatRegLog
0,[0],[0],[0],NaN


# Caso 4

In [94]:
dataset = crearRegistro()
dataset

Enter Passegerid :4
Enter Name :chavez, aminta
Enter Age: 65
Enter SibSp: 1
Enter Parch:2
Enter Ticket:Sa-21
Enter Fare:10.21
Enter Cabin:
Enter Embarked:Q
Enter passenger_class:Lower
Enter passenger_sex:F
Empty DataFrame
Columns: [PassengerId, Name, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked, passenger_class, passenger_sex]
Index: []


,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,FamilySize,IsAlone,Has_Cabin
0,4,"chavez, aminta",4,1,2,Sa-21,0,,2,0,0,1,1,1


In [95]:
ds = Eval(dataset)
print('DATOS DE SALIDA: ')
ds

Empty DataFrame
Columns: [YhatDT, YhatSVM, YhatNB, YhatRegLog]
Index: []
DATOS DE SALIDA: 


,YhatDT,YhatSVM,YhatNB,YhatRegLog
0,[1],[1],[1],NaN


# Caso 5

In [96]:
dataset = crearRegistro()
dataset

Enter Passegerid :5
Enter Name :Estrada, luis
Enter Age: 65
Enter SibSp: 1
Enter Parch:2
Enter Ticket:SA-21
Enter Fare:40.32
Enter Cabin:
Enter Embarked:C
Enter passenger_class:Middle
Enter passenger_sex:M
Empty DataFrame
Columns: [PassengerId, Name, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked, passenger_class, passenger_sex]
Index: []


,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,FamilySize,IsAlone,Has_Cabin
0,5,"Estrada, luis",4,1,2,SA-21,0,,1,1,1,4,0,1


In [97]:
ds = Eval(dataset)
print('DATOS DE SALIDA: ')
ds

Empty DataFrame
Columns: [YhatDT, YhatSVM, YhatNB, YhatRegLog]
Index: []
DATOS DE SALIDA: 


,YhatDT,YhatSVM,YhatNB,YhatRegLog
0,[0],[0],[0],NaN


# Caso 6

In [98]:
dataset = crearRegistro()
dataset

Enter Passegerid :6
Enter Name :Juarez Carlos
Enter Age: 65
Enter SibSp: 2
Enter Parch:1
Enter Ticket:SA-21
Enter Fare:40.21
Enter Cabin:S21
Enter Embarked:S
Enter passenger_class:Upper
Enter passenger_sex:M
Empty DataFrame
Columns: [PassengerId, Name, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked, passenger_class, passenger_sex]
Index: []


,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,FamilySize,IsAlone,Has_Cabin
0,6,Juarez Carlos,4,2,1,SA-21,0,S21,0,2,1,4,0,1


In [99]:
ds = Eval(dataset)
print('DATOS DE SALIDA: ')
ds

Empty DataFrame
Columns: [YhatDT, YhatSVM, YhatNB, YhatRegLog]
Index: []
DATOS DE SALIDA: 


,YhatDT,YhatSVM,YhatNB,YhatRegLog
0,[0],[0],[0],NaN


# Caso 7

In [100]:
dataset = crearRegistro()
dataset

Enter Passegerid :7
Enter Name :Jumique anabel
Enter Age: 22
Enter SibSp: 1
Enter Parch:2
Enter Ticket:SA-21
Enter Fare:20.21
Enter Cabin:
Enter Embarked:S
Enter passenger_class:Upper
Enter passenger_sex:F
Empty DataFrame
Columns: [PassengerId, Name, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked, passenger_class, passenger_sex]
Index: []


,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,FamilySize,IsAlone,Has_Cabin
0,7,Jumique anabel,1,1,2,SA-21,0,,0,2,0,4,0,1


In [101]:
ds = Eval(dataset)
print('DATOS DE SALIDA: ')
ds

Empty DataFrame
Columns: [YhatDT, YhatSVM, YhatNB, YhatRegLog]
Index: []
DATOS DE SALIDA: 


,YhatDT,YhatSVM,YhatNB,YhatRegLog
0,[1],[1],[1],NaN


# Caso 8

In [102]:
dataset = crearRegistro()
dataset

Enter Passegerid :8
Enter Name :lopez, marina
Enter Age: 54
Enter SibSp: 0
Enter Parch:0
Enter Ticket:SA01
Enter Fare:1
Enter Cabin:
Enter Embarked:S
Enter passenger_class:Lower
Enter passenger_sex:F
Empty DataFrame
Columns: [PassengerId, Name, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked, passenger_class, passenger_sex]
Index: []


,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,FamilySize,IsAlone,Has_Cabin
0,8,"lopez, marina",3,0,0,SA01,0,,0,0,0,4,0,1


In [103]:
ds = Eval(dataset)
print('DATOS DE SALIDA: ')
ds

Empty DataFrame
Columns: [YhatDT, YhatSVM, YhatNB, YhatRegLog]
Index: []
DATOS DE SALIDA: 


,YhatDT,YhatSVM,YhatNB,YhatRegLog
0,[1],[1],[1],NaN


# Caso 9

In [104]:
dataset = crearRegistro()
dataset

Enter Passegerid :9
Enter Name :franco Almyr
Enter Age: 43
Enter SibSp: 3
Enter Parch:0
Enter Ticket:SA-21
Enter Fare:20.21
Enter Cabin:
Enter Embarked:C
Enter passenger_class:Lower
Enter passenger_sex:M
Empty DataFrame
Columns: [PassengerId, Name, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked, passenger_class, passenger_sex]
Index: []


,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,FamilySize,IsAlone,Has_Cabin
0,9,franco Almyr,2,3,0,SA-21,0,,1,0,1,1,1,1


In [105]:
ds = Eval(dataset)
print('DATOS DE SALIDA: ')
ds

Empty DataFrame
Columns: [YhatDT, YhatSVM, YhatNB, YhatRegLog]
Index: []
DATOS DE SALIDA: 


,YhatDT,YhatSVM,YhatNB,YhatRegLog
0,[0],[0],[0],NaN


# Caso 10

In [106]:
dataset = crearRegistro()
dataset

Enter Passegerid :10
Enter Name :Salazar, Mr David
Enter Age: 8
Enter SibSp: 1
Enter Parch:1
Enter Ticket:Sa-21
Enter Fare:10
Enter Cabin:
Enter Embarked:S
Enter passenger_class:Upper
Enter passenger_sex:M
Empty DataFrame
Columns: [PassengerId, Name, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked, passenger_class, passenger_sex]
Index: []


,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,FamilySize,IsAlone,Has_Cabin
0,10,"Salazar, Mr David",0,1,1,Sa-21,0,,0,2,1,4,0,1


In [107]:
ds = Eval(dataset)
print('DATOS DE SALIDA: ')
ds

Empty DataFrame
Columns: [YhatDT, YhatSVM, YhatNB, YhatRegLog]
Index: []
DATOS DE SALIDA: 


,YhatDT,YhatSVM,YhatNB,YhatRegLog
0,[1],[1],[0],NaN
